In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp "/content/gdrive/My Drive/Colab Notebooks/unetplusplus/net_lung_seg.hdf5" /content/net_lung_seg.hdf5

In [ ]:
!pip install efficientnet

     |████████████████████████████████| 51kB 7.7MB/s 


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import backend
class FixedDropout(layers.Dropout):
  def _get_noise_shape(self, inputs):
    if self.noise_shape is None:
      return self.noise_shape
    symbolic_shape = backend.shape(inputs)
    noise_shape = [symbolic_shape[axis] if shape is None else shape
                   for axis, shape in enumerate(self.noise_shape)]
    return tuple(noise_shape)

In [ ]:
smooth = 1e-6


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)


def generalised_dice_loss_2d(Y_gt, Y_pred):
    smooth = 1e-6
    w = tf.reduce_sum(Y_gt)
    w = 1 / (w ** 2 + smooth)

    numerator = Y_gt * Y_pred
    numerator = w * tf.reduce_sum(numerator)
    numerator = tf.reduce_sum(numerator)

    denominator = Y_pred + Y_gt
    denominator = w * tf.reduce_sum(denominator)
    denominator = tf.reduce_sum(denominator)

    gen_dice_coef = 2 * numerator / (denominator + smooth)
    loss = tf.reduce_mean(1 - gen_dice_coef)
    return loss


def pgdl(y_true, y_pred):
    k = 2.5
    gd = generalised_dice_loss_2d(y_true, y_pred)
    loss = gd / (1 + k * (1 - gd))
    return loss


alpha = 0.9


def loss(y_true, y_pred):
    return alpha * pgdl(y_true, y_pred) + (1 - alpha) * binary_crossentropy(y_true, y_pred)

In [ ]:
from tensorflow.keras.models import load_model 
model = load_model('/content/net_lung_seg.hdf5',custom_objects={"FixedDropout":FixedDropout, "loss":loss, "dice_coef":dice_coef, "pgdl":pgdl, "generalised_dice_loss_2d":generalised_dice_loss_2d, "jacard_coef":jacard_coef, "dice_coef_loss":dice_coef_loss})

In [ ]:
!unzip '/content/gdrive/My Drive/Colab Notebooks/tbh+clahe.zip' -d '/content'

Archive:  /content/gdrive/My Drive/Colab Notebooks/tbh+clahe.zip
   creating: /content/tbh+clahe/
   creating: /content/tbh+clahe/test/
   creating: /content/tbh+clahe/test/cxrs/
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0048_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0005_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0013_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0061_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0338_1.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0070_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0083_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0047_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0080_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0266_1.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0024_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0101_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0071_0.png  
  inflating: /c

In [ ]:
BATCH_SIZE = 4
segmentation_test_path = '/content/tbh+clahe/test/'
def train_generator(batch_size, train_path, image_folder, mask_folder, aug_dict,
                    image_color_mode="rgb",
                    mask_color_mode="grayscale",
                    target_size=(512, 512),
                    seed=420):
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    seed = np.random.randint(seed)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes=[image_folder],
        class_mode=None,
        color_mode=image_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        seed=seed,
        shuffle=True)

    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes=[mask_folder],
        class_mode=None,
        color_mode=mask_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        seed=seed,
        shuffle=True)

    for (img, mask) in zip(image_generator, mask_generator):
        img, mask = normalise_train(img, mask)
        yield img, mask


def val_generator(batch_size, train_path, image_folder, mask_folder,
                  image_color_mode="rgb",
                  mask_color_mode="grayscale",
                  target_size=(512, 512)):
    image_datagen = ImageDataGenerator()
    mask_datagen = ImageDataGenerator()
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes=[image_folder],
        class_mode=None,
        color_mode=image_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False)

    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes=[mask_folder],
        class_mode=None,
        color_mode=mask_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False)

    for (img, mask) in zip(image_generator, mask_generator):
        img, mask = normalise_val(img, mask)
        yield img, mask


def normalise_train(img, mask):
    img = img / 255.0
    mask = mask / 255.0
    mask[mask > 0.5] = 1.0
    mask[mask <= 0.5] = 0.0
    return img, mask


def normalise_val(img, mask):
    img = img / 255.0
    mask = mask / 255.0
    mask[mask > 0.5] = 1.0
    mask[mask <= 0.5] = 0.0
    return img, mask

In [ ]:
from glob import glob
import os
import tensorflow as tf
from tensorflow.keras.losses import binary_crossentropy
import tensorflow.keras.backend as K

from tensorflow.keras.preprocessing.image import ImageDataGenerator
segmentation_train_path = '/content/tbh+clahe/train/'
segmentation_validation_path = '/content/tbh+clahe/train/'
segmentation_validation_image_path = '/content/tbh+clahe/validation/cxrs/'
segmentation_test_path = '/content/tbh+clahe/test/'
SEGMENTATION_IMAGE_DIR = '/content/tbh+clahe/train/cxrs/'
train_files = glob(os.path.join(SEGMENTATION_IMAGE_DIR, "*.png"))
val_files = glob(os.path.join(SEGMENTATION_IMAGE_DIR, "*.png"))

val_gen = val_generator(BATCH_SIZE,
                        segmentation_validation_path,
                        'cxrs',
                        'masks',
                        target_size=(512, 512))
model.evaluate(val_gen,
               batch_size = BATCH_SIZE,
               verbose =1,
               steps = len(val_files) / BATCH_SIZE)

Found 307 images belonging to 1 classes.
Found 307 images belonging to 1 classes.
76/76 [==============================] - 56s 231ms/step - loss: 0.0392 - binary_crossentropy: 0.0060 - dice_coef: 0.9951 - jacard_coef: 0.9903 - binary_accuracy: 0.9976


[0.03924020752310753,
 0.006039366126060486,
 0.9951063990592957,
 0.9902607202529907,
 0.9976257085800171]

In [11]:
segmentation_validation_image_path = '/content/tbh+clahe/validation/cxrs/'
segmentation_validation_path = '/content/tbh+clahe/validation/'
val_files = glob(os.path.join(segmentation_validation_image_path, "*.png"))
val_gen = val_generator(BATCH_SIZE,
                        segmentation_validation_path,
                        'cxrs',
                        'masks',
                        target_size=(512, 512))

In [12]:
model.evaluate(val_gen,
               batch_size = BATCH_SIZE,
               verbose =1,
               steps = len(val_files) / BATCH_SIZE)

Found 39 images belonging to 1 classes.
Found 39 images belonging to 1 classes.
9/9 [==============================] - 2s 195ms/step - loss: 0.0476 - binary_crossentropy: 0.0585 - dice_coef: 0.9820 - jacard_coef: 0.9648 - binary_accuracy: 0.9902


[0.047634344547986984,
 0.05845574662089348,
 0.9820415377616882,
 0.9647589921951294,
 0.9901957511901855]

In [13]:
segmentation_validation_image_path = '/content/tbh+clahe/test/cxrs/'
segmentation_validation_path = '/content/tbh+clahe/test/'
val_files = glob(os.path.join(segmentation_validation_image_path, "*.png"))
val_gen = val_generator(BATCH_SIZE,
                        segmentation_validation_path,
                        'cxrs',
                        'masks',
                        target_size=(512, 512))

In [14]:
model.evaluate(val_gen,
               batch_size = BATCH_SIZE,
               verbose =1,
               steps = len(val_files) / BATCH_SIZE)

Found 39 images belonging to 1 classes.
Found 39 images belonging to 1 classes.
9/9 [==============================] - 2s 195ms/step - loss: 0.0447 - binary_crossentropy: 0.0481 - dice_coef: 0.9837 - jacard_coef: 0.9680 - binary_accuracy: 0.9912


[0.044700220227241516,
 0.04806074872612953,
 0.9837483167648315,
 0.9680307507514954,
 0.9911890029907227]

In [ ]:
model.metrics_names

['loss', 'binary_crossentropy', 'dice_coef', 'jacard_coef', 'binary_accuracy']